In [1]:
import torch
from datasets import load_dataset
from transformers import RobertaTokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
torch.cuda.empty_cache()


c:\Users\sainithin\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
dataset = load_dataset("code_search_net", "python",cache_dir="./Datasets", trust_remote_code=True)

In [4]:
dataset.column_names

{'train': ['repository_name',
  'func_path_in_repository',
  'func_name',
  'whole_func_string',
  'language',
  'func_code_string',
  'func_code_tokens',
  'func_documentation_string',
  'func_documentation_tokens',
  'split_name',
  'func_code_url'],
 'test': ['repository_name',
  'func_path_in_repository',
  'func_name',
  'whole_func_string',
  'language',
  'func_code_string',
  'func_code_tokens',
  'func_documentation_string',
  'func_documentation_tokens',
  'split_name',
  'func_code_url'],
 'validation': ['repository_name',
  'func_path_in_repository',
  'func_name',
  'whole_func_string',
  'language',
  'func_code_string',
  'func_code_tokens',
  'func_documentation_string',
  'func_documentation_tokens',
  'split_name',
  'func_code_url']}

In [5]:
tokenizer = RobertaTokenizer.from_pretrained("Salesforce/codet5-base", cache_dir="./Models")
model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-base", cache_dir="./Models").to(device)

In [6]:
def tokenize_code(code_snippet, tokenizer):
    return tokenizer(
        code_snippet, max_length=512, truncation=True, padding='max_length', return_tensors="pt"
    )

In [7]:
def preprocess_data(batch, tokenizer):
    batch["input_ids"] = tokenize_code(batch["func_code_string"], tokenizer)["input_ids"]
    batch["labels"] = tokenize_code(batch["func_documentation_string"], tokenizer)["input_ids"]
    return batch

In [8]:
train_dataset = dataset["train"].shuffle(seed=42).select(range(10000)).map(lambda x: preprocess_data(x, tokenizer), batched=True)
eval_dataset = dataset["validation"].shuffle(seed=42).select(range(3000)).map(lambda x: preprocess_data(x, tokenizer), batched=True)

In [9]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32100, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32100, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [10]:
training_args = TrainingArguments(
    output_dir="./codebert-finetuned-roberta/results",    # output directory
    num_train_epochs=20,                   # number of epochs
    per_device_train_batch_size=4,       # batch size
    per_device_eval_batch_size=4,        # eval batch size
    warmup_steps=1000,                     # warmup steps
    weight_decay=0.01,                    # weight decay
    logging_dir="./logs",                 # logging directory
    logging_steps=1000,
    evaluation_strategy="steps",          # Evaluation after every logging step
    save_total_limit=2,                   # Keep only last two checkpoints
    save_steps=1000,                       # Save model every 500 steps
    report_to="none",
    fp16=True,
    learning_rate=5e-5                 # No reports (e.g., to wandb)
)

c:\Users\sainithin\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
# Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

In [ ]:
trainer.train()

  0%|          | 111/50000 [15:07<115:47:37,  8.36s/it]

In [13]:
model.save_pretrained("./codet5-finetuned-roberta")
tokenizer.save_pretrained("./codet5-finetuned-roberta")

('./codebert-finetuned-roberta\\tokenizer_config.json',
 './codebert-finetuned-roberta\\special_tokens_map.json',
 './codebert-finetuned-roberta\\vocab.json',
 './codebert-finetuned-roberta\\merges.txt',
 './codebert-finetuned-roberta\\added_tokens.json')

In [21]:
model = T5ForConditionalGeneration.from_pretrained("./codebert-finetuned-roberta").to(device)
tokenizer =RobertaTokenizer.from_pretrained("./codebert-finetuned-roberta")

In [22]:
def generate_documentation(code_snippet, model, tokenizer):
    # Tokenize input
    inputs = tokenizer(code_snippet, return_tensors="pt", padding=True, truncation=True).to(device)
    
    # Generate output
    outputs = model.generate(inputs["input_ids"], max_length=150, num_beams=5, early_stopping=True)
    print(outputs)
    # Decode generated text
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [23]:
code_example = """
def add_numbers(a, b):
    return a + b
"""

In [24]:
documentation = generate_documentation(code_example, model, tokenizer)
print("Generated Documentation:", documentation)

tensor([[    0, 50118,  9232,  1606,  1215,   282, 30806,  1640,   102,     6,
           741,  3256, 50118,  1437,  1437,  1437,   671,    10,  2055,   741,
         50118,     2,  1437,  1437,  1437,  1437,  1437,  1437,  1437,  1437,
          1437,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     2]], device='cuda:0')
Generated Documentation: 
def add_numbers(a, b):
    return a + b
         


In [25]:
def generate_documentation(code_snippet, model, tokenizer):
    inputs = tokenizer(code_snippet, return_tensors="pt", padding=True, truncation=True).to(device)
    outputs = model.generate(inputs["input_ids"], max_length=150, num_beams=5, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Updated code example
code_example = """
# This function adds two numbers
def add_numbers(a, b):
    return a + b
"""

documentation = generate_documentation(code_example, model, tokenizer)
print("Generated Documentation:", documentation)


Generated Documentation: 
# This function adds two numbers
def add_numbers(a, b):
    return a + b



In [6]:
import torch
from transformers import RobertaTokenizer, T5ForConditionalGeneration
from datasets import load_dataset

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained("Salesforce/codet5-base", cache_dir="./Models")
model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-base", cache_dir="./Models").to(device)

# Function to generate documentation
def generate_documentation(code_snippet):
    # Tokenize input
    inputs = tokenizer.encode(code_snippet, return_tensors="pt").to(device)
    
    # Generate output
    outputs = model.generate(inputs, max_length=150, num_beams=5, early_stopping=True)
    
    # Decode generated text
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Example code snippet
code_example = """
def add_numbers(a, b):
    return a + b
"""

# Generate documentation
documentation = generate_documentation(code_example)
print("Generated Documentation:\n", documentation)

# Save the model and tokenizer
model.save_pretrained("./code_t5_model")
tokenizer.save_pretrained("./code_t5_model")

# Load the model and tokenizer
loaded_model = T5ForConditionalGeneration.from_pretrained("./code_t5_model").to(device)
loaded_tokenizer = RobertaTokenizer.from_pretrained("./code_t5_model")

# Test the loaded model with the same code example
loaded_documentation = generate_documentation(code_example)
print("Loaded Model Generated Documentation:\n", loaded_documentation)


Generated Documentation:
 + b
   
Loaded Model Generated Documentation:
 + b
   
